## <ins>IMPORT DES DEPENDANCES ET DU DATASET</ins>

In [4]:
# !pip install torch

In [5]:
# !pip install transformers

In [6]:

import torch
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset
import tensorflow as tf

In [7]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
df = pd.read_csv('df_texte_clean.csv')

df = df.drop(["Unnamed: 0", "productid", "imageid"], axis =1)

df.head()

,prdtypecode,langue,produit_clean
0,2280,french,journal art 2001 art marche salon art asiatiqu...
1,50,french,grand stylet ergonomique bleu gamepad nintendo...
2,1280,english,peluche donald europe disneyland 2000 marionne...
3,2705,french,guerre tuques luc idées grandeur veut organise...
4,2280,french,afrique contemporaine hiver 2004 dossier japon...


In [10]:
df['prdtypecode'] = df['prdtypecode'].astype(str)
df['produit_clean'] = df['produit_clean'].astype(str)

In [11]:
labels = df['prdtypecode'].unique().tolist()
labels = [s.strip() for s in labels ]
labels

['2280',
 '50',
 '1280',
 '2705',
 '2522',
 '2582',
 '1560',
 '1281',
 '1920',
 '2403',
 '1140',
 '2583',
 '1180',
 '1300',
 '2462',
 '1160',
 '2060',
 '40',
 '60',
 '10',
 '1320',
 '1302',
 '2220',
 '2905',
 '2585',
 '1940',
 '1301']

## Pourquoi nous avons besoin d'id2label et labe2ids dans les projets PNL

Dans les tâches PNL, en particulier celles impliquant des problèmes de classification, les dictionnaires id2label et label2id sont utilisés pour mapper les étiquettes de classe (catégories) aux identifiants entiers et vice versa. Ces mappages sont essentiels pour différentes étapes du pipeline NLP, telles que le prétraitement des données, la formation du modèle et l'évaluation.

- Prétraitement des données : afin d'introduire des données textuelles dans un modèle NLP, le texte doit d'abord être tokenisé puis converti en valeurs numériques. De même, les étiquettes de classe doivent également être transformées en représentations numériques. Le dictionnaire label2id aide à convertir les étiquettes de classe d'origine en identifiants entiers.

- Formation de modèles : les modèles PNL génèrent généralement des distributions de probabilité sur les classes comme prédictions. Pendant la formation, les prédictions du modèle sont comparées aux étiquettes de vérité terrain (qui ont été converties en identifiants entiers) pour calculer la perte et optimiser les paramètres du modèle.

- Évaluation et interprétation du modèle : une fois le modèle formé, ses prédictions (sous forme d'identifiants entiers) doivent être mappées à leurs étiquettes de classe d'origine pour rendre les résultats interprétables. Le dictionnaire id2label est utilisé pour effectuer cette conversion.


Pour le modèle `BertForSequenceClassification` également, j'ai besoin de ce mappage exact de id2labels et labels2id sous forme de dictionnaire.

### Par conséquent, avant de commencer à entraîner votre modèle, créez une carte des identifiants attendus avec leurs étiquettes avec id2label et label2id :

In [12]:
for key, value in enumerate(labels):
    print(value)

2280
50
1280
2705
2522
2582
1560
1281
1920
2403
1140
2583
1180
1300
2462
1160
2060
40
60
10
1320
1302
2220
2905
2585
1940
1301


In [13]:
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [14]:
label2id

{'2280': 0,
 '50': 1,
 '1280': 2,
 '2705': 3,
 '2522': 4,
 '2582': 5,
 '1560': 6,
 '1281': 7,
 '1920': 8,
 '2403': 9,
 '1140': 10,
 '2583': 11,
 '1180': 12,
 '1300': 13,
 '2462': 14,
 '1160': 15,
 '2060': 16,
 '40': 17,
 '60': 18,
 '10': 19,
 '1320': 20,
 '1302': 21,
 '2220': 22,
 '2905': 23,
 '2585': 24,
 '1940': 25,
 '1301': 26}

In [15]:
id2label

{0: '2280',
 1: '50',
 2: '1280',
 3: '2705',
 4: '2522',
 5: '2582',
 6: '1560',
 7: '1281',
 8: '1920',
 9: '2403',
 10: '1140',
 11: '2583',
 12: '1180',
 13: '1300',
 14: '2462',
 15: '1160',
 16: '2060',
 17: '40',
 18: '60',
 19: '10',
 20: '1320',
 21: '1302',
 22: '2220',
 23: '2905',
 24: '2585',
 25: '1940',
 26: '1301'}

### Créez une nouvelle colonne pour représenter les catégories sous forme numérique

J'ai besoin d'une en-tête de colonne « étiquette » avec une valeur numérique, sinon lors de l'exécution des époques avec « trainer.train() », j'obtiendrai l'erreur ci-dessous:

```
BertForSequenceClassification ValueError: The model did not return 

```


### Ci-dessous, je le fais manuellement, mais j'aurais aussi pu le faire avec pd.factorize() comme ci-dessous

La méthode de factorisation Pandas est utilisée pour coder des variables catégorielles sous forme d'entiers. Il attribue une valeur entière unique à chaque catégorie distincte d'une série ou d'un DataFrame donné, transformant efficacement les données non numériques en valeurs numériques.

In [16]:
# AVEC PD.FACTORIZE()
# df['labels_num'] = pd.factorize(df.prdtypecode)[0]
# df.head()

In [17]:
df["labels"]=df.prdtypecode.map(lambda x: label2id[x.strip()])
df.head()

,prdtypecode,langue,produit_clean,labels
0,2280,french,journal art 2001 art marche salon art asiatiqu...,0
1,50,french,grand stylet ergonomique bleu gamepad nintendo...,1
2,1280,english,peluche donald europe disneyland 2000 marionne...,2
3,2705,french,guerre tuques luc idées grandeur veut organise...,3
4,2280,french,afrique contemporaine hiver 2004 dossier japon...,0


In [18]:
tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-medium-historic-multilingual-cased", max_length=512)

In [19]:
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-medium-historic-multilingual-cased", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-mini-historic-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, e

## Informations utiles et compréhension que vous connaissez du modèle à partir du résultat ci-dessus

**Composants du modèle :** Le modèle se compose de plusieurs composants clés, tels que :

* Intégrations (intégrations de mots, de positions et de types de jetons)

* Couches d'encodeur (12 couches dans ce cas, chacune avec des composants d'auto-attention, intermédiaires et de sortie)

* Couches de normalisation et d'abandon pour la régularisation

* Fonctions d'activation GELU utilisées dans les couches intermédiaires

**Dimensions du modèle :**

* Intégrations de mots : le modèle a une taille d'intégration de 768 dimensions et une taille de vocabulaire de 32 000 jetons.

* Intégrations de positions : le modèle peut gérer des séquences d'entrée d'une longueur maximale de 512 jetons.

* Couches d'encodeur : le modèle comporte 12 couches d'encodeur, chacune avec une taille cachée de 768 et une taille de couche intermédiaire de 3072.

* Couche de classification spécifique à une tâche : le modèle BertForSequenceClassification est conçu pour les tâches de classification de séquence. Il prend l'état caché final du jeton [CLS] et le fait passer à travers une couche linéaire et une fonction softmax pour produire des probabilités de classe. Dans ce cas, le modèle est configuré avec un nombre personnalisé d'étiquettes (NUM_LABELS) et de mappages d'étiquettes (id2label, label2id).

--------------------------

## Permet de comprendre le flux d'un texte d'entrée brut > à travers le modèle BERT pré-entraîné > et finalement de sortir de l'autre côté du modèle en tant que prédiction de classe dans le contexte de ce réglage précis spécifique à une tâche.

Dans les modèles basés sur BERT comme BertForSequenceClassification, le jeton [CLS] (abréviation de « classification ») sert de jeton spécial ajouté au début de la séquence d'entrée. Il est conçu pour être utilisé comme représentation globale de l’intégralité de la séquence d’entrée pour les tâches de classification.

Voici une description étape par étape de la manière dont le jeton [CLS] est géré lors du réglage fin d'une tâche de classification spécifique :

Tokenisation : lors du prétraitement du texte saisi, le tokenizer insère le jeton [CLS] au début de la séquence de saisie. Par exemple, si le texte d'entrée est « Ceci est un exemple de phrase. », l'entrée tokenisée ressemblera à : « [CLS] Ceci est un exemple de phrase. »

**Embeddings :** La séquence d'entrée tokenisée, y compris le jeton [CLS], est transmise via les couches d'incorporation du modèle BERT, qui convertissent les jetons en vecteurs de mots à valeurs continues.

**Couches d'encodeur :** La séquence d'entrée intégrée est ensuite traitée via les couches d'encodeur du modèle BERT, qui se composent de mécanismes d'auto-attention et de réseaux neuronaux à action directe. Au cours de ce processus, le modèle apprend à capturer les informations sémantiques et syntaxiques présentes dans la séquence d'entrée, ainsi que toutes les relations entre les jetons.

**État caché final de [CLS] :** À la fin des couches d'encodeur du modèle BERT, chaque jeton a un vecteur d'état caché correspondant. Pour le jeton [CLS], son état masqué final est utilisé comme représentation agrégée de l'intégralité de la séquence d'entrée. Ce vecteur est ensuite transmis à la couche de classification spécifique à la tâche.

**Couche linéaire :** L'état caché final du jeton [CLS] est introduit dans une couche linéaire, qui mappe le vecteur à 768 dimensions (en supposant le modèle BERT de base) à un vecteur de taille égale au nombre de classes cibles. . Il s’agit essentiellement d’une multiplication matricielle de poids suivie d’une addition de termes de biais.

**Fonction Softmax :** La sortie de la couche linéaire passe ensuite par une fonction softmax, qui convertit les valeurs de sortie brutes en probabilités de classe. La fonction softmax garantit que la somme des probabilités dans toutes les classes est égale à 1.

**Prédiction :** La classe avec la probabilité la plus élevée est choisie comme prédiction finale pour la séquence d'entrée donnée.

Lors du réglage précis d'une tâche, le modèle apprend à ajuster ses poids et ses biais en fonction des données d'entraînement et des étiquettes cibles. Cela implique la mise à jour des paramètres pré-entraînés du modèle BERT et des paramètres de la couche de classification spécifique à la tâche via des techniques de rétropropagation et d'optimisation telles que la descente de gradient. Ce processus de réglage fin permet au modèle de s'adapter à la tâche de classification spécifique et d'améliorer ses performances sur l'ensemble de données donné.

-------------------

## Dans le réglage fin spécifique à la tâche ci-dessus, tous les poids du modèle BERT pré-entraîné sont-ils modifiés au cours de ce processus de réglage fin ou seulement certains des poids sont modifiés ?


Lors du réglage précis spécifique à une tâche, tous les poids du modèle BERT pré-entraîné sont potentiellement sujets à modification, y compris les poids dans les couches d'intégration, les couches d'encodeur et la couche de classification. Cependant, la mesure dans laquelle chaque poids est modifié dépend du taux d'apprentissage, de la tâche spécifique et des données d'entraînement.

En général, peaufiner un modèle pré-entraîné comme BERT implique de mettre à jour ses poids pour mieux s'adapter à la tâche cible. Lorsque le réglage fin commence, les poids initiaux du modèle proviennent du modèle pré-entraîné, qui a déjà appris les représentations générales du langage à partir d'une tâche non supervisée à grande échelle (par exemple, la modélisation d'un langage masqué).

Lors du réglage fin, le modèle est exposé aux données et étiquettes d'entraînement spécifiques à la tâche, et les poids sont mis à jour à l'aide de la rétropropagation et de la descente de gradient.

En règle générale, le taux d'apprentissage pour le réglage fin est défini pour être inférieur au taux d'apprentissage utilisé lors de la pré-formation. En effet, le modèle pré-entraîné possède déjà une bonne compréhension du langage et le processus de réglage fin vise à apporter de petits ajustements progressifs aux poids afin d'adapter le modèle à la tâche spécifique sans perdre les précieuses connaissances linguistiques générales.

## <ins>Splitting df</ins>

In [20]:
SIZE= df.shape[0]

train_texts= list(df.produit_clean[:SIZE//2])

val_texts=   list(df.produit_clean[SIZE//2:(3*SIZE)//4 ])

test_texts=  list(df.produit_clean[(3*SIZE)//4:])

train_labels= list(df.labels[:SIZE//2])

val_labels=   list(df.labels[SIZE//2:(3*SIZE)//4])

test_labels=  list(df.labels[(3*SIZE)//4:])

In [21]:
len(train_texts)

38083

In [22]:
len(train_texts), len(val_texts), len(test_texts)

(38083, 19042, 19042)

In [23]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [24]:
class DataLoader(Dataset):
    """
    Classe Dataset personnalisée pour gérer les données texte tokenisées et les étiquettes correspondantes. Hérite de torch.utils.data.Dataset.
    """
    def __init__(self, encodings, labels):
        """
        Initialise la classe DataLoader avec des encodages et des étiquettes.

        Args:
            encodings (dict) : un dictionnaire contenant des données de texte d'entrée tokenisées (par exemple, « input_ids »,
            « token_type_ids », « attention_mask »). 
            labels (list) : une liste d’étiquettes entières pour les données de texte d’entrée.
            
        """
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Renvoie un dictionnaire contenant des données tokenisées et l'étiquette correspondante pour un index donné.

        Args:
            idx (int) : L'index de l'élément de données à récupérer.

        Returns:
            item (dict) : un dictionnaire contenant les données tokenisées et l'étiquette correspondante.
        """
        # Récupérer les données tokenisées pour l'index donné
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Ajouter l'étiquette de l'index donné au dictionnaire d'éléments
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Renvoie le nombre d'éléments de données dans l'ensemble de données.

        Returns:
            (int) : le nombre d'éléments de données dans l'ensemble de données.
        """
        return len(self.labels)

## Ci-dessus DataLoader() la ligne `item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}`

Ici, j'utilise une compréhension de dictionnaire qui construit un nouveau dictionnaire appelé « élément ». Cette ligne convertit les valeurs d'encodage associées au texte d'entrée à l'index idx donné en tenseurs PyTorch.

self.encodings est un dictionnaire contenant du texte d'entrée tokenisé avec des clés telles que « input_ids », « token_type_ids » et « attention_mask ». Ces clés représentent différents aspects du texte codé qui sont nécessaires au traitement par le modèle BERT. Les valeurs associées à ces clés sont des listes ou des tableaux d'entiers.

.items() est une méthode qui renvoie un objet de vue affichant une liste des paires clé-valeur d'un dictionnaire sous forme de tuples.

La compréhension du dictionnaire parcourt les paires clé-valeur de self.encodings avec les variables key et val. Pour chaque paire clé-valeur, il crée une nouvelle paire clé-valeur dans le dictionnaire d'éléments, où la clé reste la même et la valeur est un tenseur PyTorch créé à partir des éléments à l'index idx de la valeur d'origine.

Essentiellement, cette ligne de code convertit les parties pertinentes des codages d'entrée (par exemple, les identifiants d'entrée, les masques d'attention) à l'index idx donné en tenseurs PyTorch et les stocke dans un nouveau dictionnaire appelé item. Ce format est nécessaire pour la saisie du modèle BERT lors de la formation ou de l'évaluation.

Voici un exemple du format de sortie pour self.encodings :


```
{
     'id_entrée' : [
         [101, 2023, 2003, 1037, 2742, 102],
         [101, 1045, 2066, 5009, 2102, 102],
         [101, 2129, 2024, 2017, 1029, 102]
     ],
     'token_type_ids' : [
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0]
     ],
     'attention_masque' : [
         [1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1]
     ]
}
```

Dans cet exemple, il y a trois phrases d'entrée, chacune codée dans trois fonctionnalités différentes : input_ids, token_type_ids et attention_mask.

* input_ids : listes d'ID de jeton qui représentent le texte d'entrée. Les entiers correspondent aux jetons du vocabulaire du tokenizer.

* token_type_ids : listes d'ID de type de jeton qui indiquent le type de chaque jeton. Dans ce cas, ils valent tous 0 puisqu’il n’y a qu’une seule phrase par entrée. Dans les tâches qui nécessitent des paires de phrases, vous verrez différents ID de type de jeton pour différentes phrases.

* attention_mask : listes de valeurs binaires qui indiquent si un jeton donné doit être pris en compte (1) ou non (0). Dans cet exemple, tous les jetons sont pris en compte, donc toutes les valeurs sont 1. Les jetons de remplissage auraient une valeur de 0 dans attention_mask.

Notez que cet exemple suppose que la longueur maximale de la séquence est de 6 jetons et qu'il n'est pas nécessaire de procéder à un remplissage ou à une troncature. En pratique, vous auriez des séquences plus longues et un remplissage serait nécessaire pour que toutes les séquences d'entrée aient la même longueur.

In [25]:
train_dataloader = DataLoader(train_encodings, train_labels)

val_dataloader = DataLoader(val_encodings, val_labels)

test_dataset = DataLoader(test_encodings, test_labels)

## <ins>Training with Trainer Class</ins>

In [26]:
from transformers import TrainingArguments, Trainer

In [27]:
# !pip install scikit-learn

In [28]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    """
    Calcule l'exactitude, F1, la précision et le rappel pour un ensemble donné de prédictions.
    
    Args:
        pred (obj) : un objet contenant des attributs label_ids et prédictions.
             - label_ids (array-like) : Un tableau 1D de véritables étiquettes de classe.
             - prédictions (de type tableau) : un tableau 2D où chaque ligne représente
               une observation, et chaque colonne représente la probabilité de
               cette observation appartenant à une certaine classe.
              
    Returns:
        dict : un dictionnaire contenant les métriques suivantes :
             - Précision (float) : La proportion d'instances correctement classées.
             - F1 (float) : Le score macro F1, qui est la moyenne harmonique de précision
               et rappel. La moyenne macro calcule la métrique indépendamment pour
               chaque classe et prend ensuite la moyenne.
             - Précision (float) : La précision macro, qui est le nombre de vrais
               positifs divisés par la somme des vrais positifs et des faux positifs.
             - Recall (float) : Le rappel macro, qui est le nombre de vrais positifs
               divisé par la somme des vrais positifs et des faux négatifs.
    """
    # Extraire les vraies étiquettes de l'objet d'entrée
    labels = pred.label_ids
    
    # Obtenez les étiquettes de classe prédites en trouvant l'index de colonne avec la probabilité maximale
    preds = pred.predictions.argmax(-1)
    
    # Calculez la précision des macros, le rappel et le score F1 à l'aide de la fonction précision_recall_fscore_support de sklearn
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    
    # Calculez le score de précision à l'aide de la fonction precision_score de sklearn
    acc = accuracy_score(labels, preds)
    
    # Renvoie les métriques calculées sous forme de dictionnaire
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

## <ins>Compute metrics</ins>

argmax() : La méthode NumPy argmax() renvoie l'index de la valeur maximale le long d'un axe donné. Dans un problème de classification, nous souhaitons trouver la classe ayant la probabilité la plus élevée pour chaque observation.

(-1) : Le (-1) à l'intérieur de argmax() représente l'axe le long duquel l'opération doit être effectuée. En Python, les indices négatifs sont utilisés pour accéder aux éléments depuis la fin. Ainsi, (-1) signifie ici le dernier axe, qui correspond aux colonnes d'un tableau 2D.

In [29]:
# pip install accelerate

In [32]:
training_args = TrainingArguments(
    # Le répertoire de sortie où les prédictions du modèle et les points de contrôle seront écrits
    output_dir='./BERT_model_30epochs', 
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0 
    num_train_epochs=30,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,                
    weight_decay=0.01,
    logging_strategy='steps',
   # TensorBoard log directory                 
    logging_dir='./multi-class-logs_30epochs',            
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps", 
    fp16=True,
    load_best_model_at_end=True
)

In [33]:
trainer = Trainer(
    # le modèle pré-entraîné qui sera affiné
    model=model,
     # arguments de formation que nous avons définis ci-dessus                  
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=val_dataloader,            
    compute_metrics= compute_metrics
)

In [34]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,3.260900,3.207027,0.068690,0.011057,0.055206,0.039685
100,3.152800,3.059042,0.134335,0.016428,0.056118,0.041990
150,3.001400,2.914051,0.182859,0.032041,0.057362,0.078323
200,2.856300,2.773962,0.234587,0.063743,0.103244,0.113850
250,2.734700,2.633752,0.290621,0.118215,0.151134,0.152890
300,2.685700,2.530362,0.329010,0.132084,0.155292,0.172894
350,2.535900,2.437984,0.342611,0.149451,0.202985,0.185204
400,2.413900,2.349814,0.380737,0.170519,0.223676,0.206931
450,2.356900,2.265016,0.415923,0.195109,0.258969,0.234135
500,2.212500,2.210472,0.413980,0.207174,0.255891,0.239304


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

TrainOutput(global_step=71430, training_loss=0.20302569745866333, metrics={'train_runtime': 55963.9413, 'train_samples_per_second': 20.415, 'train_steps_per_second': 1.276, 'total_flos': 1.134443828634624e+16, 'train_loss': 0.20302569745866333, 'epoch': 30.0})

In [35]:
q=[trainer.evaluate(eval_dataset=df) for df in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.334662,0.903815,0.880764,0.900896,0.870672
val,0.744934,0.791671,0.769057,0.795196,0.756372
test,0.759670,0.787102,0.759897,0.784025,0.746561


In [36]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

In [37]:
def predict(text):
    """
    Prédit l'étiquette de classe pour un texte d'entrée donné

    Args:
        text (str) : le texte d'entrée pour lequel l'étiquette de classe doit être prédite.

    Returns:
        probs (torch.Tensor) : probabilités de classe pour le texte saisi. 
        pred_label_idx (torch.Tensor) : l'index de l'étiquette de classe prédite. 
        pred_label (str) : l'étiquette de classe prédite.
        
    """
    # Tokenisez le texte d'entrée et déplacez les tenseurs vers le GPU si disponible
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Obtenir la sortie du modèle (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    
    """ 
    Sorties d'explication : le modèle BERT renvoie un tuple contenant les logits de sortie (et éventuellement d'autres éléments en fonction
    de la configuration du modèle). Dans ce cas, les logits de sortie sont le premier élément du tuple, c'est pourquoi nous y accédons en
    utilisant outputs[0].

    Sorties[0] : il s'agit d'un tenseur contenant les logits de sortie bruts pour chaque classe. La forme du tenseur est (batch_size, num_classes)
    où batch_size est le nombre d'échantillons d'entrée (dans ce cas, 1, comme nous le prévoyons pour un seul texte d'entrée) et num_classes
    est le nombre de classes cibles.

    softmax(1) : la fonction softmax est appliquée le long de la dimension 1 (la dimension de classe) pour convertir les logits bruts en 
    probabilités de classe. Softmax normalise les logits pour qu'ils totalisent 1, les rendant interprétables comme des probabilités.
    
    """

    # Obtenez l'index de la classe avec la probabilité la plus élevée
    # argmax() trouve l'indice de la valeur maximale dans le tenseur le long d'une dimension spécifiée
    # Par défaut, si aucune dimension n'est spécifiée, il renvoie l'index de la valeur maximale dans le tenseur aplati.
    pred_label_idx = probs.argmax()

    # Mappez maintenant l'index de classe prédit à l'étiquette de classe réelle.
    # Puisque pred_label_idx est un tenseur contenant une seule valeur (l'index de classe prédit), 
    # la méthode .item() est utilisée pour extraire la valeur sous forme scalaire
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [38]:
# Testez avec un exemple de texte
text = "Aspirateur de piscine neuf."
# "Le Machine Learning lui-même évolue vers de plus en plus d'automatisation""
predict(text)

(tensor([[2.2484e-03, 8.6332e-05, 9.3165e-05, 3.3277e-04, 2.7862e-04, 6.3109e-04,
          1.2385e-04, 3.3219e-05, 1.7111e-04, 3.4279e-04, 2.5420e-05, 9.9039e-01,
          1.6158e-05, 2.9550e-04, 5.9451e-05, 1.1779e-03, 4.8428e-05, 1.1789e-04,
          5.4025e-05, 9.5388e-04, 3.9676e-04, 7.2781e-04, 4.9479e-05, 3.3089e-05,
          8.4427e-04, 4.2963e-04, 3.9526e-05]], device='cuda:0',
        grad_fn=<SoftmaxBackward0>),
 tensor(11, device='cuda:0'),
 '2583')

## <ins>Save model for inference</ins>

In [39]:
model_path = "bert-text-classification-model_3epochs"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('bert-text-classification-model_3epochs/tokenizer_config.json',
 'bert-text-classification-model_3epochs/special_tokens_map.json',
 'bert-text-classification-model_3epochs/vocab.txt',
 'bert-text-classification-model_3epochs/added_tokens.json',
 'bert-text-classification-model_3epochs/tokenizer.json')

## <ins>Re-Load saved model for inference</ins>

In [40]:
model_path = "bert-text-classification-model_3epochs"

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
nlp= pipeline("text-classification", model=model, tokenizer=tokenizer)

In [41]:
nlp("Carte pokémon Pikachu")

[{'label': '1160', 'score': 0.775137186050415}]

In [42]:
nlp("Livre la guerre du feu")

[{'label': '2403', 'score': 0.4469161927700043}]

In [43]:
nlp("Canapé 3 places en bon état")

[{'label': '2280', 'score': 0.42303213477134705}]